In [1]:
import pandas
import missingno as msno
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns

### Missing value analysis

Missing Completely at Random:  There is no pattern in the missing data on any variables. This is the best you can hope for.

Missing at Random: There is a pattern in the missing data but not on your primary dependent variables such as likelihood to recommend or SUS Scores.

Missing Not at Random: There is a pattern in the missing data that affect your primary dependent variables. For example, lower-income participants are less likely to respond and thus affect your conclusions about income and likelihood to recommend. 

Missing not at random is your worst-case scenario. Proceed with caution.

In [ ]:
 # %matplotlib notebook

In [2]:
full_training_set = pandas.read_csv('data/training_set_VU_DM_2014.csv')
print("Dataset loaded")

Dataset loaded


After loading the dataset a plot can be shown analyse the missing values. With the help of missingno a interesting plot is made.

In [ ]:
# list_missing_columns = ["srch_id"]
# for index_val, val in zip(list_of_missing_data.index,list_of_missing_data):
#     if val > 0 and index_val != "gross_bookings_usd":
#         list_missing_columns.append(index_val)


msno.matrix(full_training_set, labels=list(full_training_set))
plt.savefig('matrix.png', bbox_inches='tight')
plt.clf()

In [ ]:
msno.heatmap(full_training_set, labels=list(full_training_set))
plt.savefig('heat.png', bbox_inches='tight')
plt.clf()

In [ ]:
list(full_training_set)

In [ ]:

full_training_set.describe().to_csv('D:/Users/Thomas/Documents/GitHub/dmt_assignment_2/describe.csv')


Counting the amount of missing values in the dataset gives an insight if values should be removed Entirely or not.

In [ ]:
list_of_missing_data = full_training_set.isnull().sum()
print(list_of_missing_data)
i = 0
for index_val, val in zip(list_of_missing_data.index,list_of_missing_data):
    if val > 0:
        i += 1
print("Amount of columns with missing values: "+ str(i))

T-tests are used to figure out if the data is completely at random and thus the entire column can be removed. When there is a definitive correlation between the to predict values click_bool and/or booking_bool the missing values have to be filled in.



In [ ]:
print("average for clicking " +str(full_training_set['click_bool'].mean()))


print("average for booking " +str(full_training_set['booking_bool'].mean()))


In [ ]:
import statsmodels.api as sm
z_score, p_value = sm.stats.proportions_ztest([135, 47], [1781, 1443])

for index_val, val in zip(list_of_missing_data.index,list_of_missing_data):
    # If there is missing data we can do a t-test.
    if val > 0 and index_val != "gross_bookings_usd":
        print(index_val)
        cat1 = full_training_set[full_training_set[index_val].isnull()==True]['click_bool']
        cat2 = full_training_set[full_training_set[index_val].isnull()==False]['click_bool']
        cat1mean = cat1.mean()
        cat2mean = cat2.mean()

        if cat1mean > cat2mean:
            print("more likely to click with NULL " + str(cat1mean) + " > " + str(cat2mean))
        else:
            print("Less likely to click with NULL " + str(cat1mean) + " < " + str(cat2mean))        
#         print("average for null values: "+ str(cat1.mean()))
#         print("average for not null values: "+ str(cat2.mean()))
            
#         t2, p2 = stats.ttest_ind(cat1,cat2,equal_var=False)
#         print("t-value click boolean = " + str(t2))
#         print("p-value click boolean = " + str(p2))
        
        cat1 = full_training_set[full_training_set[index_val].isnull()==True]['booking_bool']
        cat2 = full_training_set[full_training_set[index_val].isnull()==False]['booking_bool']
        cat1mean = cat1.mean()
        cat2mean = cat2.mean()
        if cat1mean > cat2mean:
            print("more likely to book with NULL " + str(cat1mean) + " > " + str(cat2mean))
        else:
            print("Less likely to book with NULL " + str(cat1mean) + " < " + str(cat2mean))   
#         print("average for null values: "+ str(cat1mean))
#         print("average for not null values: "+ str(cat1mean))
#         t2, p2 = stats.ttest_ind(cat1,cat2,equal_var=False)
#         print("t-value booking boolean = " + str(t2))
#         print("p-value booking boolean = " + str(p2))

In [ ]:
for index_val, val in zip(list_of_missing_data.index,list_of_missing_data):
    # If there is missing data we can do a t-test.
    if val > 0 and index_val != "gross_bookings_usd":
        print(index_val)
        cat2 = full_training_set[full_training_set[index_val].isnull()==False][index_val]

        print("new average for null values: "+ str(cat2.mean()))
        print("min for null values: "+ str(cat2.min()))
        print("max for null values: "+ str(cat2.max()))
        

In [ ]:


# full_training_set['max_rate_percent_diff'] = full_training_set[['comp1_rate_percent_diff', 'comp2_rate_percent_diff', 
#                                                                 'comp3_rate_percent_diff', 'comp4_rate_percent_diff', 
#                                                                 'comp5_rate_percent_diff', 'comp6_rate_percent_diff', 
#                                                                 'comp7_rate_percent_diff', 'comp8_rate_percent_diff'] 
#                                                               ].max(axis=0)

# full_training_set['min_rate_percent_diff'] = full_training_set[['comp1_rate_percent_diff', 'comp2_rate_percent_diff', 
#                                                                 'comp3_rate_percent_diff', 'comp4_rate_percent_diff', 
#                                                                 'comp5_rate_percent_diff', 'comp6_rate_percent_diff', 
#                                                                 'comp7_rate_percent_diff', 'comp8_rate_percent_diff'] 
#                                                               ].min(axis=0)

# full_training_set['count_available'] = 0.0
# full_training_set['count_higher_rate'] = 0.0
# full_training_set['count_lower_rate'] = 0.0
# for elem in range(1,9):
#     full_training_set['count_lower_rate'] += np.where(full_training_set['comp'+str(elem)+'_rate'] == -1, 1, 0)
#     full_training_set['count_higher_rate'] += np.where(full_training_set['comp'+str(elem)+'_rate'] == 1, 1, 0)
#     full_training_set['count_available'] += np.where(full_training_set['comp'+str(elem)+'_inv'] == 1, 1, 0)

In [5]:
full_training_set.loc[:,'price_usd'] =  full_training_set['price_usd'].clip(lower=0.0, upper=1000.0)

hist = full_training_set.drop(['srch_id', 'site_id',
 'date_time',
 'random_bool',
 'promotion_flag',
 'prop_brand_bool',
 'srch_saturday_night_bool',
 'comp1_rate',
 'comp1_inv',
 'comp1_rate_percent_diff',
 'comp2_rate',
 'comp2_inv',
 'comp2_rate_percent_diff',
 'comp3_rate',
 'comp3_inv',
 'comp3_rate_percent_diff',
 'comp4_rate',
 'comp4_inv',
 'comp4_rate_percent_diff',
 'comp5_rate',
 'comp5_inv',
 'comp5_rate_percent_diff',
 'comp6_rate',
 'comp6_inv',
 'comp6_rate_percent_diff',
 'comp7_rate',
 'comp7_inv',
 'comp7_rate_percent_diff',
 'comp8_rate',
 'comp8_inv',
 'comp8_rate_percent_diff',
 'click_bool',
 'gross_bookings_usd',
 'booking_bool'],axis=1).hist(figsize=(30,24),layout=(8,7),bins=20)


plt.savefig('hist_clean.png', bbox_inches='tight')
plt.clf()

In [ ]:
hist = full_training_set.hist(figsize=(30,24),layout=(8,7),bins=20)


plt.savefig('hist.png', bbox_inches='tight')
plt.clf()

In [ ]:
# cat2 = full_training_set[full_training_set['count_available'].isnull()==False]['count_available']

# print(full_training_set['count_available'].head(5))
# print("min for null values: "+ str(cat2.min()))
# print("max for null values: "+ str(cat2.max()))

In [ ]:
import statsmodels.api as sm
z_score, p_value = sm.stats.proportions_ztest([135, 47], [1781, 1443])

for index_val, val in zip(list_of_missing_data.index,list_of_missing_data):
    # If there is missing data we can do a t-test.
    if val > 0 and index_val != "gross_bookings_usd":
        cat1 = full_training_set[full_training_set[index_val].isnull()==True]['click_bool']
        cat2 = full_training_set[full_training_set[index_val].isnull()==False]['click_bool']
        
        z_score, p_value = sm.stats.proportions_ztest([cat1.sum(), cat2.sum()], [cat1.count(), cat2.count()]) 
        print(index_val + " click proportion likelihood : " + str(stats.norm.cdf(z_score)) )
        
        cat1 = full_training_set[full_training_set[index_val].isnull()==True]['booking_bool']
        cat2 = full_training_set[full_training_set[index_val].isnull()==False]['booking_bool']

        z_score, p_value = sm.stats.proportions_ztest([cat1.sum(), cat2.sum()], [cat1.count(), cat2.count()]) 
        print(index_val + " book proportion likelihood : " + str(stats.norm.cdf(z_score)) )

In [ ]:
boxplot = full_training_set.boxplot(grid=False, rot=90, fontsize=8)
plt.savefig('box.png', bbox_inches='tight')
plt.clf()

In [ ]:
min_val = full_training_set.min()
max_val = full_training_set.max()


In [ ]:
print(min_val)

In [ ]:
full_training_set.fillna(-400)
for elem in list(full_training_set):
    if elem == "date_time":
        pass
    else:
        full_training_set[elem] = (full_training_set[elem]-min_val[elem])/(max_val[elem]-min_val[elem])

In [ ]:

full_training_set = full_training_set.mask(full_training_set < 0)
boxplot = full_training_set.drop(['srch_id', 'site_id',
 'date_time',
 'random_bool',
 'comp1_rate',
 'comp1_inv',
 'comp1_rate_percent_diff',
 'comp2_rate',
 'comp2_inv',
 'comp2_rate_percent_diff',
 'comp3_rate',
 'comp3_inv',
 'comp3_rate_percent_diff',
 'comp4_rate',
 'comp4_inv',
 'comp4_rate_percent_diff',
 'comp5_rate',
 'comp5_inv',
 'comp5_rate_percent_diff',
 'comp6_rate',
 'comp6_inv',
 'comp6_rate_percent_diff',
 'comp7_rate',
 'comp7_inv',
 'comp7_rate_percent_diff',
 'comp8_rate',
 'comp8_inv',
 'comp8_rate_percent_diff',
 'click_bool',
 'gross_bookings_usd',
 'booking_bool'],axis=1).boxplot(grid=False, rot=90, fontsize=8)
plt.savefig('box_norm_miss.png', bbox_inches='tight')
plt.clf()

In [ ]:
boxplot = full_training_set.boxplot(grid=False, rot=90, fontsize=8)
plt.savefig('box_norm.png', bbox_inches='tight')
plt.clf()

In [ ]:
# Calculate correlation

corr_matrix = full_training_set.corr()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(full_training_set.corr(), interpolation='nearest')
fig.colorbar(cax)

ax.set_xticklabels(['']+list(full_training_set))
ax.set_yticklabels(['']+list(full_training_set))

plt.savefig('corr.png', bbox_inches='tight')
plt.clf()

In [ ]:
# Generate a mask for the upper triangle
mask = np.zeros_like(corr_matrix, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(30, 22))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_matrix, mask=mask, cmap=cmap, vmax=0.4, center=0,
            square=True, linewidths=.25, cbar_kws={"shrink": .5})

plt.savefig('corr.png', bbox_inches='tight')
plt.show()